In [1]:
import math
import torch
import numpy as np
import gpytorch
import pandas as pd
from matplotlib import pyplot as plt
import random

In [2]:
filename = r'../data/olhs_run1.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Initial Design (OLHS)', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

In [3]:
#normalizing the inputs
xmeans = x_pd.mean(axis=0)
xstddv = x_pd.std(axis=0)
x_pd_normal = (x_pd - xmeans)/xstddv

x_pd = x_pd_normal

In [9]:
#normalize objective value data
y_obj_pd = y_pd.iloc[:, [0,1,2]]
ymeans = y_obj_pd.mean(axis=0)
ystddv = y_obj_pd.std(axis=0)
y_obj_pd_normal = (y_obj_pd - ymeans) / ystddv

y_pd.iloc[:, [0,1,2]] = y_obj_pd_normal

In [11]:
validation_idx = [1,7,15]

train_x_pd = x_pd.drop(validation_idx)
train_y_pd = y_pd.drop(validation_idx)

In [20]:
#make torch tensors
train_x = torch.tensor(train_x_pd.values, dtype=torch.float)
train_y1 = torch.tensor(train_y_pd['Polymer Solubility', 'mg/mL'].values, dtype=torch.float).squeeze()
train_y2 = torch.tensor(train_y_pd['Gelation Enthalpy', 'J/g'].values, dtype=torch.float).squeeze()
train_y3 = torch.tensor(train_y_pd['Shear Modulus', 'Kpa'].values, dtype=torch.float).squeeze()
train_y4 = torch.tensor(train_y_pd['Manufacturability', '--'].values, dtype=torch.long).squeeze()

In [21]:
test_x = torch.tensor(x_pd.values, dtype=torch.float)
test_y = torch.tensor(y_pd.values, dtype=torch.float).squeeze()

In [16]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
# function to optimize parameters of the regression GP -
def train_reg_gp(model, likelihood, train_x, train_y, training_iter):
   # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        if i - 1  == training_iter:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.item(),
                model.likelihood.noise.item()
            ))
        optimizer.step()

    return model, likelihood 

In [17]:
class DirichletGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# function to optimize parameters of the classification GP - 
def train_cls_gp(model, likelihood, train_x, training_iter):
   # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, likelihood.transformed_targets).sum()
        loss.backward()
        optimizer.step()

    return model, likelihood


In [22]:
reg_likl = gpytorch.likelihoods.GaussianLikelihood()
reg_model1 = ExactGPModel(train_x, train_y1, reg_likl)
reg_model2 = ExactGPModel(train_x, train_y2, reg_likl)
reg_model3 = ExactGPModel(train_x, train_y3, reg_likl)

cls_likl = gpytorch.likelihoods.DirichletClassificationLikelihood(train_y4, learn_additional_noise=False, alpha_epsilon=1e-4)
cls_model = DirichletGPModel(train_x, cls_likl.transformed_targets, cls_likl, num_classes=cls_likl.num_classes)

In [23]:
training_iter = 50

reg_model1, reg_likl1 = train_reg_gp(reg_model1, reg_likl, train_x, train_y1, training_iter)
reg_model2, reg_likl2 = train_reg_gp(reg_model2, reg_likl, train_x, train_y2, training_iter)
reg_model3, reg_likl3 = train_reg_gp(reg_model3, reg_likl, train_x, train_y3, training_iter)

In [24]:
cls_model, cls_likl = train_cls_gp(cls_model, cls_likl, train_x, training_iter)